In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from Data_Preparation import Preparation
data_set = Preparation(folder_path = 'Traders csv data') 
#from datetime import date
from backtest.data_con.data_con import DataCon
dc = DataCon('http://185.235.43.95:6263') 

[2022-05-06 02:19:29,438| INFO| create_logger()] ************ new session ************


In [2]:
def C_Leverage_Duration(data:pd.DataFrame)-> pd.DataFrame:
    """
    calculation Leverage and Duration 
    """

    data['Symbol'] = data['Symbol'].apply(lambda x : str(x))
    #-----------------------------------------------------
    for i in np.arange(1,data.shape[0]):
        if data.loc[i,'Symbol'].find('JPY') > 0:
            data['Position Returns (in pip)'] = ((data['Price'].iloc[:,1] - data['Price'].iloc[:,0])).values * 100
        elif data.loc[i,'Symbol'].find('XAU') > 0:
            data['Position Returns (in pip)'] = ((data['Price'].iloc[:,1] - data['Price'].iloc[:,0]) ).values * 100
        else:
            data['Position Returns (in pip)'] = ((data['Price'].iloc[:,1] - data['Price'].iloc[:,0]) ).values * 10000
    #----------------- for sell postion -----------------
    targhet_type_list = ['Sell Stop','Sell','Sell Limit']
    for i in np.arange(0,data.shape[0]):
        if data.loc[i,'Type'] in targhet_type_list:
            data.loc[i,'Position Returns (in pip)'] = data.loc[i,'Position Returns (in pip)'] * -1
    #----------------------------------------------------
    data['lever_profit'] = data.loc[:,'Profit'] - ((data.loc[:,['Commission','Swap']]).sum(1))
    #----------------------------------------------------
    data['Balance'] = data['Profit'].cumsum()
    #----------------------------------------------------
    for i in np.arange(1,data.shape[0]):
        if data.loc[i,'Symbol'].find('JPY') > 0:
            data.loc[i,'Leverage'] = ((data.loc[i,'lever_profit']/data.loc[i,'Position Returns (in pip)'])*(data.loc[i,'Price'][0]*100)) / data.loc[i-1,'Balance']
        elif data.loc[i,'Symbol'].find('XAU') > 0:
            data.loc[i,'Leverage'] = ((data.loc[i,'lever_profit']/data.loc[i,'Position Returns (in pip)'])*(data.loc[i,'Price'][0]*100)) / data.loc[i-1,'Balance']
        else :
            data.loc[i,'Leverage'] = ((data.loc[i,'lever_profit']/data.loc[i,'Position Returns (in pip)'])*(data.loc[i,'Price'][0]*10000)) / data.loc[i-1,'Balance']
    #----------------------------------- can add more symbol---------------
    D_lever = data.loc[:,['Time','Symbol','Leverage']].applymap(lambda x : np.nan if x == 0 else x).dropna()
    D_lever.loc[:,'Position Duration (in minute)'] = (D_lever['Time'].iloc[:,1] - D_lever['Time'].iloc[:,0]).apply(
        lambda x : np.around((x / np.timedelta64(1, 'm')),decimals=2)
    ).values
    #----------------------------------------------------------------------
    return D_lever.loc[:,['Symbol','Leverage','Position Duration (in minute)']]

In [6]:
y = data_set.Get

In [7]:
C_Leverage_Duration(y)


C:\Users\taha\AppData\Local\Temp\ipykernel_22328\2493083654.py:31: RuntimeWarning: divide by zero encountered in double_scalars
  data.loc[i,'Leverage'] = ((data.loc[i,'lever_profit']/data.loc[i,'Position Returns (in pip)'])*(data.loc[i,'Price'][0]*10000)) / data.loc[i-1,'Balance']
C:\Users\taha\AppData\Local\Temp\ipykernel_22328\2493083654.py:31: RuntimeWarning: invalid value encountered in double_scalars
  data.loc[i,'Leverage'] = ((data.loc[i,'lever_profit']/data.loc[i,'Position Returns (in pip)'])*(data.loc[i,'Price'][0]*10000)) / data.loc[i-1,'Balance']
C:\Users\taha\AppData\Local\Temp\ipykernel_22328\2493083654.py:31: RuntimeWarning: divide by zero encountered in double_scalars
  data.loc[i,'Leverage'] = ((data.loc[i,'lever_profit']/data.loc[i,'Position Returns (in pip)'])*(data.loc[i,'Price'][0]*10000)) / data.loc[i-1,'Balance']
C:\Users\taha\AppData\Local\Temp\ipykernel_22328\2493083654.py:31: RuntimeWarning: invalid value encountered in double_scalars
  data.loc[i,'Leverage'] 

,Symbol,Leverage,Position Duration (in minute)
2,EURUSD,0.379306,383.43
3,EURUSD,0.379684,24.70
4,EURUSD,0.378539,20.62
5,EURUSD,0.377830,4.22
6,EURUSD,0.343817,480.35
...,...,...,...
6684,EURUSD,0.136843,49.73
6685,EURUSD,0.442834,29.73
6686,AUDNZD,0.167161,118.22
6687,EURUSD,0.178153,64.52


In [8]:
y

,Time,Type,Volume,Symbol,Price,S/L,T/P,Time,Price,Commission,Swap,Profit,Comment,Position Returns (in pip),lever_profit,Balance,Leverage
0,2021-02-01 00:32:36,Balance,0.00,0,0.00000,0,0.00000,0,0.00000,0.00,0.0,10000.00,Deposit,0.0,10000.00,10000.00,NaN
1,2021-02-01 00:35:18,Balance,0.00,0,0.00000,0,0.00000,0,0.00000,0.00,0.0,14000.00,Deposit,0.0,14000.00,24000.00,NaN
2,2021-02-01 02:40:01,Sell,0.07,EURUSD,1.21305,0,1.21196,2021-02-01 09:03:27,1.21194,-0.56,0.0,7.77,[tp],11.1,8.33,24007.77,0.379306
3,2021-02-01 09:05:02,Sell,0.07,EURUSD,1.21239,0,1.21131,2021-02-01 09:29:44,1.21131,-0.56,0.0,7.56,[tp],10.8,8.12,24015.33,0.379684
4,2021-02-01 09:30:02,Sell,0.07,EURUSD,1.21137,0,1.21029,2021-02-01 09:50:39,1.21026,-0.56,0.0,7.77,[tp],11.1,8.33,24023.10,0.378539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6684,2022-04-08 10:15:02,Buy,0.06,EURUSD,1.08751,0,1.08723,2022-04-08 11:04:46,1.08726,-0.48,0.0,-1.50,[tp],-2.5,-1.02,32422.76,0.136843
6685,2022-04-08 10:35:02,Buy,0.12,EURUSD,1.08647,0,1.08723,2022-04-08 11:04:46,1.08726,-0.96,0.0,9.48,[tp],7.9,10.44,32432.24,0.442834
6686,2022-04-08 09:45:01,Buy,0.06,AUDNZD,1.08797,0,1.08855,2022-04-08 11:43:14,1.08856,-0.48,0.0,2.46,[tp],5.9,2.94,32434.70,0.167161
6687,2022-04-08 11:05:02,Buy,0.06,EURUSD,1.08732,0,1.08840,2022-04-08 12:09:33,1.08662,-0.48,0.0,-4.20,0,-7.0,-3.72,32430.50,0.178153
